In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

import tiktoken
tokenize = tiktoken.get_encoding('cl100k_base')

genai.configure(api_key = os.getenv("GOOGLE_API_KEY"))

In [1]:
def get_conversation_chain():

    # define the prompt
    prompt_template = """Bạn đang làm việc dưới bối cảnh phân tích kinh tế.
Bạn được cho một bài báo, bao gồm tựa đề và mô tả ngắn gọn về bài báo đó, ngoài ra bạn có
thông tin về ngày xuất bản của bài báo, và loại chủ đề mà bài báo đang đề cập tới.

Bạn cần phân tích bài báo, đưa ra tên của những thực thể (ví dụ như cổ phiếu, ngành nghề, công ty, quốc gia, tỉnh thành...)
sẽ bị ảnh hưởng trực tiếp hoặc gián tiếp bởi thông tin của bài báo.

Đưa ra theo định dạng sau:
[Entity 1]: [Explanation]
...
[Entity N]: [Explanation]

Với mỗi thực thể, ở phần Tên thực thể, hạn chế dùng dấu chấm, gạch ngang, dấu và &. Và cần ghi thêm quốc gia, địa phương cụ thể và ngành nghề của nó (nếu có). Tên chỉ nói tới một thực thể duy nhất.
Ví dụ: Ngành công nghiệp Việt Nam; Người dùng Mỹ; Ngành thép Châu Á; Ngành du lịch Hạ Long, ...

Phần giải thích mỗi thực thể, bắt buộc đánh giá số liệu được ghi, nhiều hoặc ít, tăng hoặc giảm, gấp bao nhiêu lần, ...
Cần cố gắng liên kết với nhiều thực thể khác, tuy nhiên vẫn nên phải tương đối cô đọng.
Không dùng dấu hai chấm trong phần giải thích, chỉ dùng hai chấm : để tách giữa Tên thực thể và phần giải thích.

Một ví dụ cho bài báo:

(BẮT ĐẦU VÍ DỤ)

Ngày đăng: 2025-04-07T22:51:00+07:00
Loại chủ đề: Kinh tế
Tựa đề: Nỗ lực hiện thực hóa mục tiêu thông tuyến cao tốc từ Cao Bằng đến Cà Mau 

Mô tả: Nhằm hoàn thành mục tiêu đến năm 2025 cả nước có trên 3.000 km đường cao tốc, Bộ Xây dựng, các địa phương và doanh nghiệp đang triển khai thi công 28 dự án/dự án thành phần với tổng chiều dài khoảng 1.188 km. 
Đến nay, tiến độ đa số các dự án bám sát kế hoạch, nhiều dự án đăng ký hoàn thành thông tuyến trong năm 2025. 

Danh sách thực thể sẽ bị ảnh hưởng:

Bộ Xây dựng Việt Nam: Áp lực quản lý 28 dự án, với tổng chiều dài 1188 km. Hướng tới mục tiêu 3000 km trong năm 2025, và tăng số lượng dự án gấp nhiều lần.
Chính quyền địa phương Việt Nam: Phối hợp triển khai dự án tại các tỉnh. Cần nâng cao hiệu quả quản lý ngân sách để đảm bảo tiến độ theo kế hoạch.
Doanh nghiệp xây dựng Việt Nam: Nhận khối lượng hợp đồng thi công lớn, kích thích doanh thu tăng nhanh hơn so với giai đoạn trước.
Ngành giao thông vận tải Việt Nam: Cải thiện hạ tầng cao tốc, qua đó rút ngắn thời gian di chuyển, và thúc đẩy hiệu quả logistics tăng đáng kể.
Ngành du lịch Việt Nam: Mở rộng thị trường du lịch, tạo đà tăng trưởng vượt trội. Du lịch được mở rộng tới các tỉnh miền núi, ở toàn bộ cả nước.
Ngành kinh tế Việt Nam: Tăng trưởng kinh tế, tăng cơ hội việc làm, tăng cơ hội du lịch, tăng cơ hội đầu tư vào những khu vực du lịch nghỉ dưỡng như
Nhà cung cấp vật liệu xây dựng Việt Nam: Nhu cầu xi măng, thép, bê tông theo dự án sẽ tăng trưởng cao so với mức tiêu thụ thường xuyên, từ đó nâng cao doanh thu.
Tỉnh Cao Bằng Việt Nam: Điểm đầu của dự án giao thông, góp phần phát triển kinh tế tăng trưởng. Triển vọng phát triển khu vực hơn hiện tại.
Người dân tỉnh Cao Bằng Việt Nam: Hạ tầng giao thông cải thiện, sẽ giảm thời gian di chuyển, tăng tính tiện lợi và khả năng du lịch toàn quốc.
Tỉnh Cà Mau Việt Nam: Điểm cuối của dự án, với hạ tầng giao thông hiện đại, sẽ mở rộng được thị trường du lịch kinh tế, qua đó tạo đà tăng trưởng vượt trội.
Người dân tỉnh Cà Mau Việt Nam: Hạ tầng giao thông cải thiện, sẽ giảm thời gian di chuyển, tăng cơ hội việc làm và tiếp cận dịch vụ công.

(KẾT THÚC VÍ DỤ)

Ngày đăng: {date}
Loại chủ đề: {group}
Tựa đề: {title}

Mô tả: {description}

Danh sách thực thể sẽ bị ảnh hưởng:
    """

    model = ChatGoogleGenerativeAI(model = "gemini-2.0-flash", temperature= 0.05) # create object of gemini-pro

    prompt = PromptTemplate(template = prompt_template, input_variables= ["date","group", "title", "description"])
    chain = load_qa_chain(model,chain_type="stuff", prompt = prompt)

    return chain